<a href="https://colab.research.google.com/github/ciepielajan/Multi-Class-Classification-NLP/blob/main/PREDICT%20DEVELOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREDICT DEVELOP

#### loading the necessary files to predict in develop

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# model
# https://drive.google.com/file/d/1LQzf7_SBpmee41X7ecupKFMGb8n9LXLk/view?usp=sharing
!gdown --id "1LQzf7_SBpmee41X7ecupKFMGb8n9LXLk" -O "develop_model.h5"

# config
# https://drive.google.com/file/d/11JehTWT6J_oHyVBAcdZtq8PRJ50pRKRz/view?usp=sharing
!gdown --id "11JehTWT6J_oHyVBAcdZtq8PRJ50pRKRz"  -O "develop_config.pickle"

Downloading...
From: https://drive.google.com/uc?id=1LQzf7_SBpmee41X7ecupKFMGb8n9LXLk
To: /content/develop_model.h5
11.8MB [00:00, 55.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=11JehTWT6J_oHyVBAcdZtq8PRJ50pRKRz
To: /content/develop_config.pickle
100% 451k/451k [00:00<00:00, 60.3MB/s]


In [ ]:
from keras.models import load_model
model = load_model('develop_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 100)            976800    
_________________________________________________________________
flatten (Flatten)            (None, 700)               0         
_________________________________________________________________
dense (Dense)                (None, 7)                 4907      
Total params: 981,707
Trainable params: 981,707
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import pickle
with open('develop_config.pickle', 'rb') as f:
  develop_config = pickle.load(f)

develop_config

{'labelencoder': LabelEncoder(),
 'maxlen': 7,
 'tokenizer': <keras_preprocessing.text.Tokenizer at 0x7f2b8ca88ed0>}

In [ ]:
labelencoder = develop_config["labelencoder"]
maxlen = develop_config["maxlen"]
tokenizer = develop_config["tokenizer"]

In [ ]:
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def process_text(sentence):
    sentence = re.sub('[A-Za-z0-9]+@[a-zA-z].[a-zA-Z]+', '', sentence)  # maile
    sentence = re.sub('(http[s]*:[/][/])[a-zA-Z0-9]+', '', sentence)  # linki
    sentence = re.sub(r"<[^>]+>", " ", sentence) # remove html tag
    sentence = re.sub(r"[^a-zA-Z\s]", "", sentence)  # remove punctuations and numbers   
    sentence = re.sub(r"\b[A-Za-z]{1}\b", "", sentence)  # remove single characters

    sentence = re.sub("^\s+|\s+$", "", sentence, flags=re.UNICODE) # Remove spaces both in the BEGINNING and in the END of a string:
    sentence = " ".join(re.split("\s+", sentence, flags=re.UNICODE))  # Remove ONLY DUPLICATE spaces:
    sentence = sentence.lower()
    
    return sentence

#### Predict

In [ ]:
# wszysktie etykiety
print(labelencoder.classes_.tolist())

['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork', 'SearchScreeningEvent']


In [ ]:
# x = "PLAY <dasf>M</html> my 32 favorite music"
# x = "find my favorite book"
# x = "check the 'Red Hot Chili Peppers' discography"
# x = "play the 'Red Hot Chili Peppers' discography"
x = "how will it be outside today" #@param {type:"string"}

x = process_text(x)
x = tokenizer.texts_to_sequences([x])
x = pad_sequences(x, padding="post", truncating="post", maxlen=maxlen)
predicted_lstm_val = np.argmax(model.predict(x), axis=-1)
y_pred = labelencoder.inverse_transform(predicted_lstm_val)[0]
y_proba = round(model.predict_proba(x)[0][predicted_lstm_val[0]]*100,3)
print(y_pred, "- probability :",y_proba  ,"%")

GetWeather na : 99.983 %


#### rozkład prawdopodobieństa wg wszsytkich etykiet

In [ ]:
model.predict_proba(x)
classes_proba = [ round(i * 100,4) for i in model.predict_proba(x)[0] ]
classes_id = labelencoder.transform(labelencoder.classes_)
classes_list = labelencoder.classes_.tolist()

import pandas as pd
pd.DataFrame((zip(classes_list, classes_proba)), index=classes_id, columns=("classes_list", "classes_proba" ))

,classes_list,classes_proba
0,AddToPlaylist,0.0005
1,BookRestaurant,0.0012
2,GetWeather,99.9830
3,PlayMusic,0.0076
4,RateBook,0.0024
5,SearchCreativeWork,0.0037
6,SearchScreeningEvent,0.0016
